In [229]:
import os
os.chdir('../src/')

import pandas
import numpy
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


import utilities
from importlib import reload
import utilities
utilities = reload(utilities)

In [204]:
MAX_ROWS = 1000
df_raw_train = pandas.read_csv("../data/raw/train_ver2.csv", nrows=MAX_ROWS)
df_raw_test = pandas.read_csv("../data/raw/test_ver2.csv", nrows=MAX_ROWS)

In [205]:
df_raw_train.shape

(1000, 48)

In [206]:
df_raw_train.iloc[0]

fecha_dato                      2015-01-28
ncodpers                           1375586
ind_empleado                             N
pais_residencia                         ES
sexo                                     H
age                                     35
fecha_alta                      2015-01-12
ind_nuevo                              0.0
antiguedad                               6
indrel                                 1.0
ult_fec_cli_1t                         NaN
indrel_1mes                            1.0
tiprel_1mes                              A
indresi                                  S
indext                                   N
conyuemp                               NaN
canal_entrada                          KHL
indfall                                  N
tipodom                                1.0
cod_prov                              29.0
nomprov                             MALAGA
ind_actividad_cliente                  1.0
renta                              87218.1
segmento   

In [207]:
df_raw_train.isnull().sum()

fecha_dato                  0
ncodpers                    0
ind_empleado                1
pais_residencia             1
sexo                        1
age                         0
fecha_alta                  1
ind_nuevo                   1
antiguedad                  0
indrel                      1
ult_fec_cli_1t            999
indrel_1mes                 1
tiprel_1mes                 1
indresi                     1
indext                      1
conyuemp                 1000
canal_entrada               1
indfall                     1
tipodom                     1
cod_prov                    1
nomprov                     1
ind_actividad_cliente       1
renta                     165
segmento                    1
ind_ahor_fin_ult1           0
ind_aval_fin_ult1           0
ind_cco_fin_ult1            0
ind_cder_fin_ult1           0
ind_cno_fin_ult1            0
ind_ctju_fin_ult1           0
ind_ctma_fin_ult1           0
ind_ctop_fin_ult1           0
ind_ctpp_fin_ult1           0
ind_deco_f

In [211]:
columns = ['fecha_dato', 'ncodpers', 'sexo', 'antiguedad', 'age', 'renta'] + list(df_raw_train.columns[24:-4])
df_train = df_raw_train[columns].copy()

# Cleaning antiguedad
df_train = utilities.impute_median(df_train, 'antiguedad')
# Cleaning age
df_train = utilities.impute_median(df_train, 'age')
# cleaning sex
df_train['sexo'] = df_train['sexo'].fillna('H')
# clean income imputing median
df_train['renta'] = df_train['renta'].fillna(df_train['renta'].dropna().median())

# make dummies for sexo
df_train['sexo'] = pandas.get_dummies(df_train['sexo'], drop_first=True)

In [212]:
df_train.isnull().sum()

fecha_dato           0
ncodpers             0
sexo                 0
antiguedad           0
age                  0
renta                0
ind_ahor_fin_ult1    0
ind_aval_fin_ult1    0
ind_cco_fin_ult1     0
ind_cder_fin_ult1    0
ind_cno_fin_ult1     0
ind_ctju_fin_ult1    0
ind_ctma_fin_ult1    0
ind_ctop_fin_ult1    0
ind_ctpp_fin_ult1    0
ind_deco_fin_ult1    0
ind_deme_fin_ult1    0
ind_dela_fin_ult1    0
ind_ecue_fin_ult1    0
ind_fond_fin_ult1    0
ind_hip_fin_ult1     0
ind_plan_fin_ult1    0
ind_pres_fin_ult1    0
ind_reca_fin_ult1    0
ind_tjcr_fin_ult1    0
ind_valo_fin_ult1    0
dtype: int64

In [213]:
df_train.columns

Index(['fecha_dato', 'ncodpers', 'sexo', 'antiguedad', 'age', 'renta',
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1'],
      dtype='object')

In [214]:
X_train = df_train.loc[:, ['sexo', 'antiguedad', 'age', 'renta']]
y_train = df_train['ind_ahor_fin_ult1']

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [215]:
clf.predict(X_train)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [216]:
product_columns = [c for c in df_train.columns if c.startswith('ind') and c.endswith('ult1')]

models = []

known_variable = []
for p in product_columns:
    X_train = df_train.loc[:, ['sexo', 'antiguedad', 'age', 'renta']]
    y_train = df_train[p]

    clf = RandomForestClassifier(max_depth=2, random_state=0)
    clf.fit(X_train, y_train)
    models.append(clf)
    known_variable = X_train.columns

In [230]:
df_test = df_raw_test[known_variable].copy()
df_test.isnull().sum()

# Cleaning antiguedad
# df_test = impute_median(df_test, 'antiguedad')
# Cleaning age
# df_test = impute_median(df_test, 'age')
# cleaning sex
df_test['sexo'] = df_test['sexo'].fillna('H')
# clean income imputing median
df_test = utilities.impute_median(df_test, 'renta')

# make dummies for sexo
df_test['sexo'] = pandas.get_dummies(df_test['sexo'], drop_first=True)

In [219]:
models[3].predict(df_test[known_variable])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [220]:
collect_predictions = []
for m in models:
    collect_predictions.append(m.predict(df_test[known_variable]))

In [221]:
df_predictions = pandas.DataFrame(numpy.array(collect_predictions).T, columns=product_columns)
df_predictions['ncodpers'] = df_raw_test['ncodpers']
result = []

for df_p in df_predictions.iterrows():
    cur_row = [df_p[1]['ncodpers'], ' '.join(list(df_p[1][df_p[1] == 1].keys()))]
    result.append(cur_row)

In [195]:
pandas.DataFrame(result, columns=['ncodpers', 'added_products']).to_csv("../data/final/submission.csv", index=False)